In [32]:
from tensorflow.keras.datasets import reuters
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from numpy import argmax

plt.rcParams['font.family'] = 'Malgun Gothic'

In [22]:
# 데이터를 train과 test로 나눔
# 가장 많이 사용된 단어 1000개 사용
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)

C:\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [23]:
# 패딩
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

In [24]:
# train과 validation 분리
val_n = round(len(x_train)*0.3)

x_val = x_train[:val_n] # train의 30%를 validation으로 사용
x_train = x_train[val_n:]
y_val = y_train[:val_n] # train의 30%를 validation으로 사용
y_train = y_train[val_n:]

In [25]:
# 랜덤 포레스트 분류기에 100개의 의사결정 트리를 사용한다.
forest=RandomForestClassifier(n_estimators=100)

# 단어 묶음을 벡터화한 데이터와 정답 데이터를 가지고 학습을 시작한다.
forest.fit(x_train, y_train)

RandomForestClassifier()

In [26]:
rf_preds = forest.predict(x_val)

print('accuracy: {:0.3f}'.format(accuracy_score(y_val, rf_preds)))
print('Precision: {:0.3f}'.format(precision_score(y_val, rf_preds, average='macro')))
print('recall: {:0.3f}'.format(recall_score(y_val, rf_preds, average='macro')))

accuracy: 0.479
Precision: 0.386
recall: 0.109


C:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
from sklearn.model_selection import GridSearchCV

params = { 'min_samples_split' : [4, 8, 16, 20],
           'min_samples_leaf' : [4, 8, 12, 18],
           'max_depth' : [4, 6, 8, 10, 12]
         }

rf_clf = RandomForestClassifier(random_state = 2, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(x_train, y_train)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최적 정확도: {:.3f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 12, 'min_samples_leaf': 4, 'min_samples_split': 4}
최적 정확도: 0.464


n_estimators : 결정트리의 개수<br>
	개수가 늘어날수록 성능은 높아지지만 시간이 오래걸림  
min_samples_split : 노드를 분할하기 위한 최소한의 샘플 데이터 수<br>
	과적합 제어하는데 사용  
min_samples_leaf : leaf노드가 되기 위해 필요한 최소한의 샘플 데이터 수<br>
	과적합 제어하는데 사용  
max_features : 최적 분할을 위해 고려해야 할 최대 feature 개수<br>
max_depth : 트리의 최대 깊이<br>
	깊이가 깊어지면 과적합될 수 있음  
max_leaf_nodes : leaf노드의 최대개수

In [28]:
# gridsearch로 찾은 최적 하이퍼파라미터 사용
rf_clf_new = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 12,
                                min_samples_leaf = 4,
                                min_samples_split = 4,
                                random_state = 2,
                                n_jobs = -1)
rf_clf_new.fit(x_train, y_train)
pred_new = rf_clf_new.predict(x_val)
print('accuracy: {:.3f}'.format(accuracy_score(y_val, pred_new)))
print('Precision: {:0.3f}'.format(precision_score(y_val, pred_new, average='macro')))
print('recall: {:0.3f}'.format(recall_score(y_val, pred_new, average='macro')))

accuracy: 0.463
Precision: 0.179
recall: 0.055


C:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
rf_preds_t = rf_clf_new.predict(x_test)

print('accuracy: {:0.3f}'.format(accuracy_score(y_test, rf_preds_t)))
print('Precision: {:0.3f}'.format(precision_score(y_test, rf_preds_t, average='macro')))
print('recall: {:0.3f}'.format(recall_score(y_test, rf_preds_t, average='macro')))

accuracy: 0.483
Precision: 0.154
recall: 0.069


C:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
# 모델 사용결과
xhat_idx = np.random.choice(x_test.shape[0], 15)
xhat = x_test[xhat_idx]
yhat = rf_clf_new.predict(xhat)

num=0
for i in range(15):
    print('True : ' + str(y_test[xhat_idx[i]]) + ', Predict : ' + str(yhat[i]))
    if str(y_test[xhat_idx[i]]) == str(yhat[i]):
        num+=1

print("15개 중 모델이 일치한 것의 개수 : ", num)

True : 1, Predict : 3
True : 3, Predict : 3
True : 4, Predict : 4
True : 24, Predict : 4
True : 19, Predict : 3
True : 20, Predict : 20
True : 9, Predict : 4
True : 4, Predict : 3
True : 25, Predict : 4
True : 21, Predict : 4
True : 3, Predict : 3
True : 8, Predict : 3
True : 11, Predict : 4
True : 4, Predict : 4
True : 18, Predict : 4
15개 중 모델이 일치한 것의 개수 :  5
